In [5]:
!pip install import-ipynb

  Using cached import-ipynb-0.1.3.tar.gz (4.0 kB)
  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=d1a0d2624570572b573475be60e8b389e98ce3cb4b4b2680e77247743bc10ac6
  Stored in directory: /Users/bobo/Library/Caches/pip/wheels/06/7e/ad/1cb03e935234186825cefc7e2c8f3451b4f654b5bc72232a7b
Successfully built import-ipynb


In [ ]:
import import_ipynb

In [3]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
import csv
import datetime
import time
import os
from selenium import webdriver

import matplotlib.pyplot as plt
import collections
from openpyxl import load_workbook

import math
import statistics
import numpy as np
import pandas as pd


In [21]:
def set_driver(url):

    ser = Service(DRIVER_PATH)
    options = webdriver.ChromeOptions()  # Set up Chinese
    # options.add_argument('--headless')
    options.add_argument('lang=zh_CN.UTF-8')  # Replacement of head
    options.add_argument(
        'User-Agent="Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36"')
    # options.add_argument("--disable-infobars")
    options.add_argument("--headless")
    # PROXY = "11.456.448.110:8080"
    # options.add_argument('--proxy-server=%s' % PROXY)
    options.add_argument("--window-size=1920,1080");
    options.add_argument("--no-sandbox");
    options.add_argument("--disable-extensions");
    options.add_argument("--dns-prefetch-disable");
    options.add_argument("--disable-gpu");
    # options.setPageLoadStrategy(PageLoadStrategy.NORMAL);
    driver = webdriver.Chrome(
        service=ser,
        options=options)

    driver.get(url)
    print("finish setup driver")
    return driver


def init_filename(source, subject):
    # /呱吉/PTT.csv
    # /呱吉/FB.csv
    # ...
    folder = f"/{source}/"
    return subject + folder


def write_file():
    out_filename = ""
    with open(out_filename, 'a') as out_file:
        for line in in_file:

            out_file.write(parsed_line)


def init_csv(file_name):

    with open(file_name, 'w+') as out_file:
        write = csv.writer(out_file)
        header = ['Source', 'Title', 'Link', 'Date', 'Summary', 'Text', 'Like']
        # PTT, 呱吉吃大便, http://, '2021-11-17', XXXX, XXXXX, 23
        write.writerow(header)


def string_to_datetime(s):
    return datetime.datetime.strptime(s, '%Y/%m/%d')


def generate_ytr_name_list(filename="Youtuber名單 - 道歉.csv"):
    ytr_list = []
    with open(filename, 'r') as f:
        rows = csv.reader(f)
        for row in rows:
            if row[1] == "":
                pass
            if row[1] == "頻道(官方帳號)":
                continue
            ytr_name = []
            ytr_name.append(row[1].split("\n")[0])

            ytr_name.extend(row[3].replace("、", "\n").split("\n"))
            ytr_list.append(list(dict.fromkeys(ytr_name)))
    return ytr_list


In [15]:
DRIVER_PATH = '/home/bobo/Desktop/bobo/ML_final/bobo/crawlers/chromedriver'
# DRIVER_PATH = "/Users/bobo/OneDrive - 台灣微軟/0. bobo/Codes/stratechery crawler/chromedriver"
apple_news_url = "https://tw.appledaily.com/search/阿神/"


In [6]:
def scroll_to_button(driver):
    reached_page_end = False
    last_height = driver.execute_script("return document.body.scrollHeight")

    while not reached_page_end:
        driver.find_element(By.XPATH, '//body').send_keys(Keys.END)
        time.sleep(5)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if last_height == new_height:
            reached_page_end = True
        else:
            last_height = new_height
    print("reach end")
    # driver.quit()


def crawl_page(driver, filename):
    contianer_list = driver.find_element(
        By.CLASS_NAME, "article-container").find_elements(By.CLASS_NAME, "flex-feature")
    
    if contianer_list == []:
        print(contianer_list)
        print("is empty")
        return


    print("len: ", len(contianer_list))

    filename = filename
    with open(filename, 'a+') as f:
        write = csv.writer(f)

        header = ['Source', 'Title', 'Link', 'Date', 'Summary', 'Text', 'Like']
        dic = {i: "" for i in header}

        for i in contianer_list:
            dic['Source'] = "apple_news"
            dic['Title'] = i.find_element(
                By.CLASS_NAME, 'storycard-headline').text.replace("  ", " ")
            dic['Link'] = i.find_element(
                By.TAG_NAME, 'a').get_attribute("href")
            dic['Date'] = i.find_element(By.CLASS_NAME, 'timestamp').text
            dic['Summary'] = ""
            dic['Text'] = ""
            dic['Like'] = 0

            line = dic.values()
            write.writerow(line)


In [7]:
# tag search
def crawl_apple_news(path, tag_url):
    driver = set_driver(tag_url)

    scroll_to_button(driver)

    time.sleep(10)

    crawl_page(driver, filename=path+"apple_news.csv")

    driver.quit()



In [ ]:
ytr_list = generate_ytr_name_list("Youtuber名單 - 工作表3.csv")
ytr_list

In [31]:
base_path = "data/"
if not os.path.exists(base_path):
    os.makedirs(base_path)

for ytr_name in ytr_list:
    
    if ytr_name == ['']:
        continue
        
    print(ytr_name)
    folder_path = base_path + ytr_name[0] + "/"
    
    # if folder not exists, create
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)    
    
    # if file exists, continue, else, create
    if os.path.exists(folder_path+"apple_news.csv"):
        pass
    else:
        init_csv(folder_path+"apple_news.csv")
        for name in ytr_name:
            apple_url = "https://tw.appledaily.com/search/"+name+"/"
            try:
                crawl_apple_news(folder_path, apple_url)
            except Exception as e:
                print("error :", e)
                continue    
        
    print("*"*20)
    print()

['阿滴英文', '阿滴', '都省瑞']
********************

['啾啾鞋']
********************

['亮生活 / Bright Side', '亮生活']
********************

['理科太太', '陳映彤']
********************

['Taiwan Bar', '台灣吧']
********************

['那些電影教我的事 Lessons from Movies']
********************

['志祺七七 X 圖文不符', '志祺']
********************

['柴鼠兄弟 ZRBros', '柴鼠兄弟']
********************

['許伯&簡芝—倉鼠人', '許伯&簡芝－倉鼠人', '許伯', '簡芝', '倉鼠人']
********************

['Cheap', 'cheap']
********************

['窮奢極慾']
********************

['三個字SunGuts']
********************

['維思維WeisWay', '維思維', 'WeisWay']
********************

['營養健身葛格Peeta', 'Peeta', '葛格']
********************

['蒼藍鴿的醫學天地', '蒼藍鴿', '蒼藍教主']
********************

['SKIMMY 你的網路閨蜜', 'skimmy你的網路閨密', 'skimmy']
********************

['劉芒']
********************

['Dr. Ivan 6', 'Dr.Ivan']
********************

['冏冏電台', '冏冏子', '冏星人', '冏冏']
********************

['超級歪SuperY', '超級歪']
********************

['快樂大學Happiness University', '快樂大學']
********************

['昆蟲擾西吳沁婕Dee the 

## crawl web text

In [32]:
from bs4 import BeautifulSoup
import scrapy

In [33]:
def get_filename_from_folder(folder_path, ext=""):
    file_name_list = []
    for file_name in os.listdir(folder_path):
        name = os.path.join(folder_path + "/"+ file_name)
        
        # get file extension
        file_ext = os.path.splitext(file_name)[1]
        # if user give ext and the file extension is not the same, pass
        if ext and file_ext != ext:
            continue
            
        file_name_list.append(name)
        
    return sorted(file_name_list)

In [34]:
driver = set_driver(apple_news_url)

finish setup driver


In [35]:
# print apple news which hasn't finish crawling
for idx, folder_path in enumerate(get_filename_from_folder("./data")):
    try:
        for file_path in get_filename_from_folder(folder_path, ".csv"):
                    # if not apple news, continue
            if not file_path.split("/")[-1] == "apple_news.csv":
                continue

            df = pd.read_csv(file_path)

            if df.shape[0] < 2 or df["Text"].notnull().sum() > 2:
                continue
            else:
                print(file_path)
    except Exception as e:
        print("error: ", e)


./data/Astor/apple_news.csv
./data/BugCat Capoo/apple_news.csv
./data/Celeste Wu 大沛/apple_news.csv
./data/Cheap/apple_news.csv
./data/Dinter/apple_news.csv
./data/Hello Catie/apple_news.csv
./data/I'm Charlie/apple_news.csv
./data/It‘s Jcnana 蒨蒨/apple_news.csv
./data/Jessica Lin/apple_news.csv
./data/Jinnytty 企鵝妹 윰찌 /apple_news.csv
./data/J爸/apple_news.csv
./data/Kimi紀卜心/apple_news.csv
./data/MASAの料理ABC/apple_news.csv
./data/Meg Lu/apple_news.csv
./data/NanaQ/apple_news.csv
./data/NyoNyoTV妞妞TV/apple_news.csv
./data/Onion Man/apple_news.csv
./data/RICO/apple_news.csv
./data/SKIMMY 你的網路閨蜜/apple_news.csv
./data/SelinaCheng Life/apple_news.csv
./data/Taiwan Bar/apple_news.csv
error:  [Errno 20] Not a directory: './data/Untitled.ipynb'
./data/三個字SunGuts/apple_news.csv
./data/六指淵 Huber/apple_news.csv
./data/冏冏電台/apple_news.csv
./data/劉芒/apple_news.csv
./data/千千進食中/apple_news.csv
./data/南瓜 Pumpkin/apple_news.csv
./data/厭世甜點店/apple_news.csv
./data/反正我很閒/apple_news.csv
./data/吃貨豪豪 HowHowEat/app

In [ ]:
for idx, folder_path in enumerate(get_filename_from_folder("./data")):
    for file_path in get_filename_from_folder(folder_path, ".csv"):
        # if not apple news, continue
        if not file_path.split("/")[-1] == "apple_news.csv":
            continue
        
        print(file_path)
        df = pd.read_csv(file_path)
        
        # if already have data, continue
        if df.shape[0] < 2 or df["Text"].notnull().sum() > 1:
            continue
        else:
            pass

            
        for row, link in enumerate(df.loc[:, "Link"]):
            print(row, link)
            driver.get(link)
            time.sleep(1)
            # soup = BeautifulSoup(driver.page_source, "html.parser")
            
            text = driver.find_element(By.ID, "articleBody").text
            
            df.loc[row,"Text"] = text
            
        df.to_csv(file_path)
        print("*"*20)
        time.sleep(5)

./data/AC草影水族/apple_news.csv
./data/Astor/apple_news.csv
0 https://tw.appledaily.com/life/20210803/AREXLUNQFRESXGKBLXKWWNILQA/
1 https://tw.appledaily.com/entertainment/20210611/I6CAIKUTPZDXJOTGE6G5RT3ZPE/
2 https://tw.appledaily.com/international/20200325/NYWUWLYENIJOM2VX2OLETQ33CQ/
3 https://tw.appledaily.com/international/20200213/OVYYM5JXQWPZHRTR2SROFKWNYU/
4 https://tw.appledaily.com/international/20200117/4VYAR6Z3KPUQ6WC3YLQY2LEADQ/
5 https://tw.appledaily.com/international/20200114/2LW3RAYEW5LXPRNDQXWREFKVUI/
6 https://tw.appledaily.com/international/20190908/HQQ5QRMXMRFJSJSIFUTBKJDLOA/
7 https://tw.appledaily.com/supplement/20180124/B6PEG6GUFTCEXY5LLV6AISX57U/
8 https://tw.appledaily.com/adcontent/20171115/32YFIPWJYEQHDIZMB7YPC3U3HE/
9 https://tw.appledaily.com/property/20171006/2MNQTFOPAPO4BTFMHE4SXLEOSI/
10 https://tw.appledaily.com/international/20170215/OUHFYP4UEOUNSXW4MWUOT4E7BU/
11 https://tw.appledaily.com/property/20170205/3JBSY3TIQRE5GOPLUA56NKZI6M/
12 https://tw.apple